In [1]:
# Import libraries.
%pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from keras import layers
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pandas'

In [ ]:

# load and visualize the data
dataset = pd.read_csv('insurance.csv')
dataset.head()

In [ ]:
# split data for training and testing
y = dataset.expenses
data_features = ['age', 'sex', 'bmi', 'children', 'smoker']
X = dataset[data_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [ ]:
# check for missing values
cols_missing = [col for col in train_X.columns if train_X[col].isnull().any()]
print(cols_missing)

In [ ]:
# check for categorical values
categ = (train_X.dtypes == 'object')
categ_cols = list(categ[categ].index)
print("Columns with categorical values are:")
print(categ_cols)
print(val_X)

In [ ]:
# process categorical values
from sklearn.preprocessing import OneHotEncoder

my_encoder = OneHotEncoder(handle_unknown='ignore', sparse = False)

OH_cols_train = pd.DataFrame(my_encoder.fit_transform(train_X[categ_cols]))
OH_cols_val = pd.DataFrame(my_encoder.transform(val_X[categ_cols]))

OH_cols_train.index = train_X.index
OH_cols_val.index = val_X.index

num_train_X = train_X.drop(categ_cols, axis = 1)
num_val_X = val_X.drop(categ_cols, axis = 1)


OH_train_X = pd.concat([num_train_X, OH_cols_train], axis = 1)
OH_val_X = pd.concat([num_val_X, OH_cols_val], axis = 1)

OH_train_X.columns = OH_train_X.columns.astype(str)
OH_val_X.columns = OH_val_X.columns.astype(str)

OH_train_X.rename(columns = {'2' : 'no smoking'}, inplace = True)
OH_val_X.rename(columns = {'2' : 'no smoking'}, inplace = True)
OH_train_X.rename(columns = {'3' : 'yes smoking'}, inplace = True)
OH_val_X.rename(columns = {'3' : 'yes smoking'}, inplace = True)

OH_train_X.rename(columns = {'1' : 'male'}, inplace = True)
OH_val_X.rename(columns = {'1' : 'male'}, inplace = True)
OH_train_X.rename(columns = {'0' : 'female'}, inplace = True)
OH_val_X.rename(columns = {'0' : 'female'}, inplace = True)


print(OH_val_X)




In [ ]:
for data in OH_train_X.select_dtypes("object"):
    OH_train_X[data], _ = OH_train_X[data].factorize()

# for data in OH_train_X.select_dtypes("float64"):
#     OH_train_X[data], _ = OH_train_X[data].factorize

print(OH_train_X)
discrete_features = (OH_train_X.dtypes == int) | (OH_train_X.dtypes == 'int64')
print(discrete_features)

In [ ]:
# Mutual Information check
from sklearn.feature_selection import mutual_info_regression
def make_mi_score(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name = 'MI Scores', index = X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_score(OH_train_X, train_y, discrete_features)
print(mi_scores)

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")



plt.figure(dpi = 100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
my_model= GradientBoostingRegressor()
my_model.fit(OH_train_X, train_y)
predict = my_model.predict(OH_val_X)
score = mean_absolute_error(val_y, predict)
print(score)

